In [ ]:
import pickle, numpy
from sklearn.model_selection import train_test_split
from keras.datasets import cifar10

## Load the data from Keras dataset<br>
Returns two tuples:<br>
X, X_test: uint8 array of RGB image data with shape (num_samples, 3, 32, 32).<br>
y, y_test: uint8 array of category labels (integers in range 0-9) with shape (num_samples,).

In [ ]:
# the data, split between train and test sets
(X,y), (X_test, y_test) = cifar10.load_data()

In [ ]:
# split training set into training and validation sets:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

## Write data to project filesystem

In [ ]:
with open('cifar-10-tf-train.pkl', 'wb') as f:
    pickle.dump((X_train, y_train), f, protocol=pickle.HIGHEST_PROTOCOL)

with open('cifar-10-tf-valid.pkl', 'wb') as f:
    pickle.dump((X_valid, y_valid), f, protocol=pickle.HIGHEST_PROTOCOL)

with open('cifar-10-tf-test.pkl', 'wb') as f:
    pickle.dump((X_test, y_test), f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config

## Insert Cloud Object Storage credentials 

1. Select the Data icon (upper right, icon with 1 and 0s)
2. Select Connections
3. Select the cell below and then select <b>Insert to code</b> for <b>Connection to project COS</b> to insert credentials
4. Rename the credentials (which are usually called credentials_1) to cos_credentials.   YOU WILL GET ERRORS IF YOU DO NOT DO THIS!

In [ ]:
service_endpoint = cos_credentials['url']  

In [ ]:
auth_endpoint = cos_credentials['iam_url']  

In [ ]:
cos = ibm_boto3.client('s3',
                      ibm_api_key_id=cos_credentials['api_key'],
                      ibm_service_instance_id=cos_credentials['resource_instance_id'],
                      ibm_auth_endpoint=auth_endpoint,
                      config=Config(signature_version='oauth'),
                      endpoint_url=service_endpoint)

## Create bucket name -- I usually append my name to the end (i.e. cifar10-tutorials-joel)

In [ ]:
bucket = "cifar10-tutorials-test-blb"

## This call may fail if bucket has already been created -- that's fine, simply continue executing the next cells

In [ ]:
cos.create_bucket(Bucket=bucket)

In [ ]:
files = ['cifar-10-tf-train.pkl',
         'cifar-10-tf-valid.pkl',
         'cifar-10-tf-test.pkl']

In [ ]:
for file in files:
    print('Uploading data {}...', format(file))
    cos.upload_file(Filename=file, Bucket=bucket, Key=file)

In [ ]:
cos.list_objects(Bucket=bucket)

## The number of the image we wish to test from the validation set

In [ ]:
image_number = 110

In [ ]:
with open("cifar-10-tf-valid.pkl", "rb") as f:
  cifar_test_data=pickle.load(f)
cifar_test_data[0][image_number].tolist()

In [ ]:
print (cifar_test_data[1][110])